In [1]:
import os
os.environ["THEANO_FLAGS"] = "device=cpu,floatX=float32"

In [2]:
import os
import sys
import traceback
import functools
from collections import defaultdict
import glob2 as glob
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import seaborn as sbn
import deepdish as dd
import theano
import theano.tensor as T
import lasagne
from IPython import display
from tqdm import tqdm
import neural_networks as nn
from music_utils import generateSequence
from nnet_utils import get_next_batch_rnn
%matplotlib inline

import pdb

/usr/local/lib/python2.7/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
# load data from each class
datapath = '/Users/rafaelvalle/Desktop/datasets/Piano/'
glob_folder = '/Users/rafaelvalle/Desktop/datasets/Piano/*/'
glob_file = '*.npy'
data_dict = defaultdict(list)
n_pieces = 40
for folderpath in glob.glob(glob_folder):
    composer = os.path.basename(os.path.normpath(folderpath))
    filepaths = glob.glob(os.path.join(os.path.join(datapath, composer), '*.npy'))
    for filepath in np.random.choice(filepaths, n_pieces):
        data_dict[composer].append(np.load(filepath))

In [4]:
d_batch_size = 512
g_batch_size = 512
min_len = 32
max_len = 64
n_features = data_dict[data_dict.keys()[0]][0].shape[1]
n_conditions = len(data_dict.keys())
n_labels = n_conditions + 1
temperature = 1

In [5]:
def output_nonlinearity(data, temperature=1):
    return T.clip(lasagne.nonlinearities.softmax(lasagne.nonlinearities.linear(data / temperature)), 1e-7, 1 - 1e-7)
    
softmax_temperature = functools.partial(output_nonlinearity, temperature=temperature)

In [6]:
d_specs = {'batch_size': d_batch_size,
           'input_shape': (None, None, n_features),
           'mask_shape': (None, None),
           'n_output_units': n_labels,
           'n_units': 32,
           'n_hidden': 32,
           'grad_clip': 100.,
           'init': lasagne.init.HeUniform(),
           'non_linearities': (
              lasagne.nonlinearities.tanh,  # feedforward
              lasagne.nonlinearities.tanh,  # feedbackward
              softmax_temperature),  # apply sotfmax with temperature
           'learning_rate': 0.1,           
          }

g_specs = {'batch_size': g_batch_size,
           'input_shape': (None, None, n_features),
           'noise_shape': (None, None, n_features),
           'cond_shape': (None, None, n_conditions),
           'mask_shape': (None, None),
           'n_output_units': n_features,
           'n_units': 32,
           'n_hidden': 32 ,
           'grad_clip': 100.,
           'init': lasagne.init.HeUniform(),
           'non_linearities': (
              lasagne.nonlinearities.tanh,  # feedforward
              lasagne.nonlinearities.tanh,  # feedbackward
              lasagne.nonlinearities.rectify),  # apply sotfmax with temperature
           'learning_rate': 0.1,
          }

In [7]:
# declare theano variables
d_in_X = T.ftensor3('ddata')
d_in_M = T.imatrix('dismask')
g_in_D = T.ftensor3('gdata')
g_in_Z = T.ftensor3('noise')
g_in_C = T.ftensor3('condition')
g_in_M = T.imatrix('genmask')

In [ ]:
gate_parameters = lasagne.layers.recurrent.Gate(
    W_in=lasagne.init.Orthogonal(), 
    W_hid=lasagne.init.Orthogonal(),
    b=lasagne.init.Constant(0.),
    nonlinearity=lasagne.nonlinearities.sigmoid)
cell_parameters = lasagne.layers.recurrent.Gate(
    W_in=lasagne.init.Orthogonal(), 
    W_hid=lasagne.init.Orthogonal(),
    W_cell=None, b=lasagne.init.Constant(0.),
    nonlinearity=lasagne.nonlinearities.tanh)

In [8]:
def build_discriminator(params):
    # input layers
    l_in = lasagne.layers.InputLayer(shape=d_specs['input_shape'], name='d_in') 
    l_mask = lasagne.layers.InputLayer(shape=d_specs['mask_shape'], name='d_mask')

    # recurrent layers for bidirectional network
    l_forward = lasagne.layers.RecurrentLayer(
        l_in, d_specs['n_units'], grad_clipping=d_specs['grad_clip'],
        W_in_to_hid=d_specs['init'], W_hid_to_hid=d_specs['init'],
        nonlinearity=d_specs['non_linearities'][0], only_return_final=True, mask_input=l_mask)
    l_backward = lasagne.layers.RecurrentLayer(
        l_in, d_specs['n_units'], grad_clipping=d_specs['grad_clip'],
        W_in_to_hid=d_specs['init'], W_hid_to_hid=d_specs['init'],        
        nonlinearity=d_specs['non_linearities'][1], only_return_final=True, mask_input=l_mask,
        backwards=True)

    # concatenate output of forward and backward layers
    l_concat = lasagne.layers.ConcatLayer([l_forward, l_backward])

    # output layer
    l_out = lasagne.layers.DenseLayer(
        l_concat, num_units=d_specs['n_output_units'], nonlinearity=d_specs['non_linearities'][2])

    class Discriminator:
        def __init__(self, l_in, l_mask, l_out):
            self.l_in = l_in
            self.l_mask = l_mask
            self.l_out = l_out
            
    return Discriminator(l_in, l_mask, l_out)

In [ ]:
def build_generator_lstm(params):
    # input layers
    l_in = lasagne.layers.InputLayer(shape=g_specs['input_shape'], input_var=g_in_D, name='g_in')
    l_noise = lasagne.layers.InputLayer(shape=g_specs['noise_shape'], input_var=g_in_Z, name='g_noise')
    l_cond = lasagne.layers.InputLayer(shape=g_specs['cond_shape'], input_var=g_in_C, name='g_cond')
    l_mask = lasagne.layers.InputLayer(shape=g_specs['mask_shape'], input_var=g_in_M, name='g_mask')
    
    # recurrent layers for bidirectional network
    l_forward_data = lasagne.layers.recurrent.LSTMLayer(
        l_in, g_specs['n_units'], mask_input=l_mask, 
        ingate=gate_parameters, forgetgate=gate_parameters,
        cell=cell_parameters, outgate=gate_parameters,
        learn_init=True, grad_clipping=g_specs['grad_clip'], only_return_final=True)
    l_forward_noise = lasagne.layers.recurrent.LSTMLayer(
        l_noise, g_specs['n_units'], mask_input=l_mask, 
        ingate=gate_parameters, forgetgate=gate_parameters,
        cell=cell_parameters, outgate=gate_parameters,
        learn_init=True, grad_clipping=g_specs['grad_clip'], only_return_final=True)
    l_forward_cond = lasagne.layers.recurrent.LSTMLayer(
        l_cond, g_specs['n_units'], mask_input=l_mask, 
        ingate=gate_parameters, forgetgate=gate_parameters,
        cell=cell_parameters, outgate=gate_parameters,
        learn_init=True, grad_clipping=g_specs['grad_clip'], only_return_final=True)

    l_backward_data = lasagne.layers.recurrent.LSTMLayer(
        l_in, g_specs['n_units'], mask_input=l_mask, 
        ingate=gate_parameters, forgetgate=gate_parameters,
        cell=cell_parameters, outgate=gate_parameters,
        learn_init=True, grad_clipping=g_specs['grad_clip'], only_return_final=True,
        backwards=True)
    l_backward_noise = lasagne.layers.recurrent.LSTMLayer(
        l_noise, g_specs['n_units'], mask_input=l_mask, 
        ingate=gate_parameters, forgetgate=gate_parameters,
        cell=cell_parameters, outgate=gate_parameters,
        learn_init=True, grad_clipping=g_specs['grad_clip'], only_return_final=True,
        backwards=True)
    l_backward_cond = lasagne.layers.recurrent.LSTMLayer(
        l_cond, g_specs['n_units'], mask_input=l_mask, 
        ingate=gate_parameters, forgetgate=gate_parameters,
        cell=cell_parameters, outgate=gate_parameters,
        learn_init=True, grad_clipping=g_specs['grad_clip'], only_return_final=True,
        backwards=True)

    # sum linearities of data and condition on forward and and backward recurrent layers
    l_forward_sum = lasagne.layers.ElemwiseSumLayer([l_forward_data, l_forward_noise, l_forward_cond])
    l_backward_sum = lasagne.layers.ElemwiseSumLayer([l_backward_data, l_backward_noise, l_backward_cond])

    # apply nonlinearity to forward and backward recurrent layers
    l_forward_nonlinearity = lasagne.layers.NonlinearityLayer(l_forward_sum, 
        nonlinearity=g_specs['non_linearities'][0])
    l_backward_nonlinearity = lasagne.layers.NonlinearityLayer(l_backward_sum, 
        nonlinearity=g_specs['non_linearities'][1])

    # concatenate output of forward and backward layers
    l_concat = lasagne.layers.ConcatLayer(
        [l_forward_nonlinearity, l_backward_nonlinearity])

    # output layer where time is collapsed into one dimension
    l_out = lasagne.layers.DenseLayer(
        l_concat, num_units=g_specs['n_output_units'], nonlinearity=g_specs['non_linearities'][2])        

    class Generator:
        def __init__(self, l_in, l_noise, l_cond, l_mask, l_out):
            self.l_in = l_in
            self.l_noise = l_noise
            self.l_cond = l_cond
            self.l_mask = l_mask
            self.l_out = l_out

    return Generator(l_in, l_noise, l_cond, l_mask, l_out)

In [9]:
def build_generator(params):
    # input layers
    l_in = lasagne.layers.InputLayer(shape=g_specs['input_shape'], input_var=g_in_D, name='g_in')
    l_noise = lasagne.layers.InputLayer(shape=g_specs['noise_shape'], input_var=g_in_Z, name='g_noise')
    l_cond = lasagne.layers.InputLayer(shape=g_specs['cond_shape'], input_var=g_in_C, name='g_cond')
    l_mask = lasagne.layers.InputLayer(shape=g_specs['mask_shape'], input_var=g_in_M, name='g_mask')

    # recurrent layers for bidirectional network
    l_forward_data = lasagne.layers.RecurrentLayer(
        l_in, g_specs['n_units'], grad_clipping=g_specs['grad_clip'],
        W_in_to_hid=g_specs['init'], W_hid_to_hid=g_specs['init'],
        nonlinearity=lasagne.nonlinearities.linear, only_return_final=True, mask_input=l_mask)
    l_forward_noise = lasagne.layers.RecurrentLayer(
        l_noise, g_specs['n_units'], b=lasagne.init.Constant(0.), grad_clipping=g_specs['grad_clip'], 
        W_in_to_hid=g_specs['init'], W_hid_to_hid=g_specs['init'],        
        nonlinearity=lasagne.nonlinearities.linear, only_return_final=True, mask_input=l_mask)
    l_forward_cond = lasagne.layers.RecurrentLayer(
        l_cond, g_specs['n_units'], b=lasagne.init.Constant(0.), grad_clipping=g_specs['grad_clip'], 
        W_in_to_hid=g_specs['init'], W_hid_to_hid=g_specs['init'],        
        nonlinearity=lasagne.nonlinearities.linear, only_return_final=True, mask_input=l_mask)
    l_backward_data = lasagne.layers.RecurrentLayer(
        l_in, g_specs['n_units'], grad_clipping=g_specs['grad_clip'],
        W_in_to_hid=g_specs['init'], W_hid_to_hid=g_specs['init'],        
        nonlinearity=lasagne.nonlinearities.linear, only_return_final=True,
        backwards=True, mask_input=l_mask)
    l_backward_noise = lasagne.layers.RecurrentLayer(
        l_noise, g_specs['n_units'], b=lasagne.init.Constant(0.), grad_clipping=g_specs['grad_clip'],
        W_in_to_hid=g_specs['init'], W_hid_to_hid=g_specs['init'],        
        nonlinearity=lasagne.nonlinearities.linear, only_return_final=True,
        backwards=True, mask_input=l_mask)    
    l_backward_cond = lasagne.layers.RecurrentLayer(
        l_cond, g_specs['n_units'], b=lasagne.init.Constant(0.), grad_clipping=g_specs['grad_clip'],
        W_in_to_hid=g_specs['init'], W_hid_to_hid=g_specs['init'],        
        nonlinearity=lasagne.nonlinearities.linear, only_return_final=True,
        backwards=True, mask_input=l_mask)

    # sum linearities of data and condition on forward and and backward recurrent layers
    l_forward_sum = lasagne.layers.ElemwiseSumLayer([l_forward_data, l_forward_noise, l_forward_cond])
    l_backward_sum = lasagne.layers.ElemwiseSumLayer([l_backward_data, l_backward_noise, l_backward_cond])

    # apply nonlinearity to forward and backward recurrent layers
    l_forward_nonlinearity = lasagne.layers.NonlinearityLayer(l_forward_sum, 
        nonlinearity=g_specs['non_linearities'][0])
    l_backward_nonlinearity = lasagne.layers.NonlinearityLayer(l_backward_sum, 
        nonlinearity=g_specs['non_linearities'][1])

    # concatenate output of forward and backward layers
    l_concat = lasagne.layers.ConcatLayer(
        [l_forward_nonlinearity, l_backward_nonlinearity])

    # output layer where time is collapsed into one dimension
    l_out = lasagne.layers.DenseLayer(
        l_concat, num_units=g_specs['n_output_units'], nonlinearity=g_specs['non_linearities'][2])    


    class Generator:
        def __init__(self, l_in, l_noise, l_cond, l_mask, l_out):
            self.l_in = l_in
            self.l_noise = l_noise
            self.l_cond = l_cond
            self.l_mask = l_mask
            self.l_out = l_out

    return Generator(l_in, l_noise, l_cond, l_mask, l_out)

In [10]:
def build_training(discriminator, generator, d_specs, g_specs):
    # get variables from discrimiator and generator
    d_params = lasagne.layers.get_all_params(discriminator.l_out, trainable=True)
    g_params = lasagne.layers.get_all_params(generator.l_out, trainable=True)
    
    d_labels = T.fmatrix('d_label')
    g_labels = T.fmatrix('g_label')    

    # G(z)
    g_z = lasagne.layers.get_output(generator.l_out,
                                    inputs={generator.l_in: g_in_D,
                                            generator.l_noise: g_in_Z,
                                            generator.l_cond: g_in_C,
                                            generator.l_mask: g_in_M})

    # create proper G(z) mask
    g_in_M_idx = g_in_M.sum(axis=1)
    g_in_M_alt = theano.clone(g_in_M) 
    g_in_M_alt = T.set_subtensor(g_in_M_alt[T.arange(g_in_M_alt.shape[0]), g_in_M_idx], 
                                 T.ones_like(g_in_M_alt[:, 0]))

    # create proper G(z)
    g_in_D_alt = theano.clone(g_in_D)
    g_in_D_alt = T.set_subtensor(g_in_D_alt[T.arange(g_in_D_alt.shape[0]), g_in_M_idx], 
                                 g_z)

    # D(G(z))
    d_g_z = lasagne.layers.get_output(
        discriminator.l_out,         
        inputs={discriminator.l_in: g_in_D_alt, 
                discriminator.l_mask: g_in_M_alt})

    g_loss = lasagne.objectives.categorical_crossentropy(1 - d_g_z, g_labels)        
    g_loss = g_loss.mean()

    g_updates = lasagne.updates.adagrad(g_loss, g_params, g_specs['learning_rate'])
    g_train_fn = theano.function(inputs=[g_in_D, g_in_Z, g_in_C, g_in_M, g_labels],
                                 outputs=g_loss, 
                                 updates=g_updates)
    g_sample_fn = theano.function(inputs=[g_in_D, g_in_Z, g_in_C, g_in_M],
                                  outputs=g_z)

    # D(x)
    d_x = lasagne.layers.get_output(discriminator.l_out,
                                    inputs={discriminator.l_in: d_in_X, 
                                            discriminator.l_mask: d_in_M})
  
    d_x_loss = lasagne.objectives.categorical_crossentropy(d_x, d_labels)
    d_g_z_loss = lasagne.objectives.categorical_crossentropy(d_g_z, g_labels)
    d_loss = d_x_loss + d_g_z_loss
    d_loss = d_loss.mean()    
    
    d_updates = lasagne.updates.adagrad(d_loss, d_params, d_specs['learning_rate'])
    d_train_fn = theano.function(
        inputs=[d_in_X, d_in_M, g_in_D, g_in_Z, g_in_C, g_in_M, d_labels, g_labels],
        outputs=[d_loss, d_x_loss, d_g_z_loss, d_x, d_g_z, g_in_D_alt], 
        updates=d_updates) 

    d_predict_fn = None

#   d_predict_fn = theano.function(
#        inputs=[d_in_X, d_in_M],
#        outputs=lasagne.layers.get_output(discriminator[2],
#                                          inputs={discriminator[0]: d_in_X, 
#                                                  discriminator[1]: d_in_M}))

    return d_train_fn, d_predict_fn, g_train_fn, g_sample_fn

In [11]:
def sample_generator_batch(batch_size, inputs, conds, masks):
    excerpt = np.random.permutation(len(inputs))[:batch_size]
    # create random noise
    noises = np.random.normal(size=inputs[excerpt].shape).astype('float32') 
    # create labels
    lbls = np.zeros((batch_size, conds.shape[2]), dtype=np.float32)
    lbls[:, n_conditions-1] = 1
    
    return inputs[excerpt], noises, conds[excerpt], masks[excerpt], lbls


def sample_data(data, batch_size, min_len, max_len, clip=False):
    encoding = defaultdict(lambda _: 0)
    i = 0
    for k in data.keys():
        encoding[k] = i
        i += 1        
    
    pieces_per_lbl = int(batch_size / len(data.keys()))

    while True:
        inputs, targets, labels, conds, masks = [], [], [], [], []
        for k, lbl in encoding.items():
                pieces = np.random.choice(data[k], pieces_per_lbl)
                for piece in pieces:
                    start_idx = np.random.randint(0, piece.shape[0] - max_len - 1)
                    piece_data = piece[start_idx: start_idx+max_len]
                    mask_size = np.random.randint(min_len, max_len)
                    if clip:
                        piece_data[mask_size:] = 0
                    target = piece[start_idx+max_len]
                    label = np.zeros(len(encoding)+1, dtype=np.float32) #+1 for generator label
                    label[encoding[k]] = 1

                    cond = np.zeros((max_len, len(encoding)), dtype=np.float32) #+1 for generator label
                    cond[:, encoding[k]] = 1                

                    mask = np.zeros(max_len, dtype=np.int32)
                    mask[:mask_size] = 1            

                    inputs.append(piece_data)
                    targets.append(target)
                    labels.append(label)
                    conds.append(cond)
                    masks.append(mask)
                
        inputs = np.array(inputs, dtype=np.float32)
        targets = np.array(targets, dtype=np.float32)
        labels = np.array(labels, dtype=np.float32)
        conds = np.array(conds, dtype=np.float32)        
        masks = np.array(masks, dtype=np.int32)
        
        shuffle_ids = np.random.randint(0, len(inputs), len(inputs))
        yield inputs[shuffle_ids], targets[shuffle_ids], labels[shuffle_ids], conds[shuffle_ids], masks[shuffle_ids]

In [12]:
discriminator = build_discriminator(d_specs)
generator = build_generator_lstm(g_specs)

In [13]:
d_train_fn, d_predict_fn, g_train_fn, g_sample_fn = build_training(discriminator, generator, d_specs, g_specs)

In [14]:
data_iter = sample_data(data_dict, d_batch_size, min_len, max_len)

In [15]:
# Pre training
n_d_iterations_pre = 0
d_losses_pre = []

folderpath = 'piano_gan_pre_{}_dlr_{}_glr_{}_bs_{}_temp_{}'.format(
    n_d_iterations_pre, d_specs['learning_rate'], g_specs['learning_rate'], d_batch_size, temperature)

if not os.path.exists(os.path.join('images', folderpath)):
    os.makedirs(os.path.join('images', folderpath))

for i in range(n_d_iterations_pre):
    # use same data for discriminator and generator
    d_X, _, d_L, d_C, d_M = data_iter.next()
    g_Z = np.random.normal(size=d_X.shape).astype('float32')     
    g_L = np.zeros((g_Z.shape[0], n_labels), dtype=np.float32)
    g_L[:, -1] = 1
    g_D, g_C, g_M = d_X, d_C, d_M
    d_loss, d_x_loss, d_g_z_loss, d_x, d_g_z, g_in_D_alt = d_train_fn(d_X, d_M, g_D, g_Z, g_C, g_M, d_L, g_L)
    d_losses_pre.append(d_loss)
    if i == (n_d_iterations_pre -1):
        fig, axes = plt.subplots(5, 2, figsize=(16, 20))
        axes[0, 0].set_title('D Label')
        sbn.heatmap(d_L.T, ax=axes[0, 0])
        axes[0, 1].set_title('G Label')
        sbn.heatmap(g_L.T, ax=axes[0, 1])

        axes[1, 0].set_title('D(x)')
        sbn.heatmap(d_x.T, ax=axes[1, 0])
        axes[1, 1].set_title('D(G(z))')        
        sbn.heatmap(d_g_z.T, ax=axes[1, 1])

        axes[2, 0].set_title('G(z) : 0')
        axes[2, 1].set_title('G(z) : 1')
        axes[3, 0].set_title('G(z) : 2')
        axes[3, 1].set_title('G(z) : 3')        
        sbn.heatmap(g_in_D_alt[0].T, ax=axes[2, 0]).invert_yaxis()
        sbn.heatmap(g_in_D_alt[1].T, ax=axes[2, 1]).invert_yaxis()
        sbn.heatmap(g_in_D_alt[2].T, ax=axes[3, 0]).invert_yaxis()
        sbn.heatmap(g_in_D_alt[3].T, ax=axes[3, 1]).invert_yaxis()        

        axes[4, 0].set_title('Loss(d)')                        
        axes[4, 0].plot(d_losses_pre)

In [16]:
## store losses
d_losses = []
g_losses = []

n_iterations = int(1e4)
n_d_iterations = 1
n_g_iterations = 1
epoch = 100
for iteration in tqdm(range(n_iterations)):
    for i in range(n_d_iterations):
        d_X, d_T, d_L, d_C, d_M = data_iter.next()
        g_Z = np.random.normal(size=d_X.shape).astype('float32')
        g_L = np.zeros((g_Z.shape[0], n_labels), dtype=np.float32)
        g_L[:, -1] = 1
        g_D, g_C, g_M = d_X, d_C, d_M
        d_loss, d_x_loss, d_g_z_loss, d_x, d_g_z, g_in_D_alt = d_train_fn(d_X, d_M, g_D, g_Z, g_C, g_M, d_L, g_L)
        d_losses.append(d_loss)
        
    for i in range(n_g_iterations):
        d_X, d_T, d_L, d_C, d_M = data_iter.next()
        g_Z = np.random.normal(size=d_X.shape).astype('float32')     
        g_D, g_C, g_M = d_X, d_C, d_M
        
        g_loss = g_train_fn(g_D, g_Z, g_C, g_M, g_L)
        g_losses.append(g_loss)

    if iteration % epoch == 0:
        fig, axes = plt.subplots(5, 2, figsize=(32, 32))
        axes[0, 0].set_title('D Label')
        sbn.heatmap(d_L.T, ax=axes[0, 0])
        axes[0, 1].set_title('G Label')
        sbn.heatmap(g_L.T, ax=axes[0, 1])
        
        axes[1, 0].set_title('D(x)')
        sbn.heatmap(d_x.T, ax=axes[1, 0])
        axes[1, 1].set_title('D(G(z))')        
        sbn.heatmap(d_g_z.T, ax=axes[1, 1])
        
        axes[2, 0].set_title('G(z) : 0')
        axes[2, 1].set_title('G(z) : 1')
        axes[3, 0].set_title('G(z) : 2')
        axes[3, 1].set_title('G(z) : 3')        
        sbn.heatmap(g_in_D_alt[0].T, ax=axes[2, 0]).invert_yaxis()
        sbn.heatmap(g_in_D_alt[1].T, ax=axes[2, 1]).invert_yaxis()
        sbn.heatmap(g_in_D_alt[2].T, ax=axes[3, 0]).invert_yaxis()
        sbn.heatmap(g_in_D_alt[3].T, ax=axes[3, 1]).invert_yaxis()        
        
        axes[4, 0].set_title('Loss(d)')                        
        axes[4, 0].plot(d_losses)
        axes[4, 1].set_title('Loss(g)')                        
        axes[4, 1].plot(g_losses)
        
        fig.savefig('images/{}/iteration_{}'.format(folderpath, iteration))
        plt.close('all') 
    
        display.clear_output(wait=True)

100%|██████████| 10000/10000 [6:16:42<00:00,  1.85s/it]
